In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.tools.plotting import lag_plot
%matplotlib inline
%precision 4
from ipywidgets import widgets
from IPython.display import display
plt.style.use('ggplot')
import pystan
import util
pd.set_option('max_rows', 1000)
pd.set_option('max_columns', 50)

In [2]:
dir (util
    )

['__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 'create_engine',
 'exec_sql',
 'golf_db',
 'np',
 'np_from_sql',
 'np_to_sql',
 'pd',
 'pd_from_sql',
 'pq',
 'scale_plot_size']

In [3]:
categories = ['Off the Tee', 'Approach to the Green', 'Around the Green', 'Putting']
def plot_series(category):
    sqlTxt = '''
    select date, strokes_gained_baseline 
      from strokes 
     where player_id = 30925 
       and year = 2016 
       and strokes_gained_category LIKE %s
       order by date;
    '''
    df = util.pd_from_sql(sqlTxt, [category])
    df.set_index(['date'])
    sg = df[(df['date'] > '2016-5-4') & (df['date'] <= '2016-6-30')]['strokes_gained_baseline']
    # lag_plot(sg)
    window = 100
    rolling = sg.rolling(window).mean().as_matrix()[window:]
    std_roll = sg.rolling(window).std().as_matrix()[window:]
    sg = sg[window:]
    plt.plot(range(0, len(sg)), sg)
    foo = plt.errorbar(range(0, len(rolling)), rolling, std_roll, linestyle=None, marker=".")
    plt.axis([0, len (sg), -1.5, 1.5])
plot_series(categories[3])

NameError: global name 'scale_plot_size' is not defined

In [115]:
foo = range(0, 10)
foo[1:]

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
code = """
data {
  int N;
  int y[N];
}
model {
 
}

"""
data = {
    'N': len(sg),
    'y': sg.tolist()
}

fit = pystan.stan(model_code=code, data=data, iter=1000, chains=4)
means = fit.get_posterior_mean()
print fit

789